In [1]:
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import sklearn
import pprint
import itertools
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
from data import WN18

In [2]:
class Config:
    n_epochs = 30
    batch_size = 150
    embed_dim = 200
    activation = tf.nn.relu
    dropout_rate = 0.2

In [3]:
"""
e: entity
s: subject
p: predicate
o: object
"""
def glance_dict(d, n=5):
    return dict(itertools.islice(d.items(), n))


def read_triples(path):
    triples = []
    with open(path, 'rt') as f:
        for line in f.readlines():
            s, p, o = line.split()
            triples += [(s.strip(), p.strip(), o.strip())]
    return triples


def load_triple():
    WN18.download()
    triples_tr = read_triples('../data/WN18/wn18/train.txt')
    triples_va = read_triples('../data/WN18/wn18/valid.txt')
    triples_te = read_triples('../data/WN18/wn18/test.txt')
    triples_all = triples_tr + triples_va + triples_te
    return triples_all, triples_tr, triples_va, triples_te


def build_vocab(triples):
    params = {}
    e_set = {s for (s, p, o) in triples} | {o for (s, p, o) in triples}
    p_set = {p for (s, p, o) in triples}
    params['e_vocab_size'] = len(e_set)
    params['p_vocab_size'] = len(p_set)
    e2idx = {e: idx for idx, e in enumerate(sorted(e_set))}
    p2idx = {p: idx for idx, p in enumerate(sorted(p_set))}
    return e2idx, p2idx, params


def build_multi_label(triples_tr):
    sp2o = {}
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        if (s,p) not in sp2o:
            sp2o[(s,p)] = [o]
        else:
            if o not in sp2o[(s,p)]:
                sp2o[(s,p)].append(o)
    return sp2o


def get_y(triples_tr, e2idx, p2idx, sp2o):
    y = []
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        temp = np.zeros([len(e2idx)])
        temp[sp2o[(s,p)]] = 1.
        y.append(temp)
    y = np.asarray(y)
    return y


def next_train_batch(triples_tr, e2idx, p2idx, sp2o):
    for i in range(0, len(triples_tr), Config.batch_size):
        _triples_tr = triples_tr[i: i+Config.batch_size]
        x_s = np.asarray([e2idx[s] for (s, p, o) in _triples_tr], dtype=np.int32)
        x_p = np.asarray([p2idx[p] for (s, p, o) in _triples_tr], dtype=np.int32)
        y = get_y(_triples_tr, e2idx, p2idx, sp2o)
        yield (x_s, x_p, y)


def train_input_fn(triples_tr, e2idx, p2idx, s2p2o):
    dataset = tf.data.Dataset.from_generator(
        lambda: next_train_batch(sklearn.utils.shuffle(triples_tr),
                                 e2idx,
                                 p2idx,
                                 s2p2o),
        (tf.int32, tf.int32, tf.float32),
        (tf.TensorShape([None]),
         tf.TensorShape([None]),
         tf.TensorShape([None, len(e2idx)])))
    iterator = dataset.make_one_shot_iterator()
    x_s, x_p, y = iterator.get_next()
    return {'s': x_s, 'p': x_p}, y

In [4]:
def o_next_batch(eval_triples,
                 e2idx, 
                 p2idx):
    for (s, p, o) in tqdm(eval_triples, total=len(eval_triples), ncols=70):
        s_idx, p_idx = e2idx[s], p2idx[p]
        yield np.atleast_1d(s_idx), np.atleast_1d(p_idx)


def o_input_fn(eval_triples,
               e2idx, 
               p2idx):
    dataset = tf.data.Dataset.from_generator(
        lambda: o_next_batch(eval_triples,
                             e2idx, 
                             p2idx),
        (tf.int32, tf.int32),
        (tf.TensorShape([None,]),
         tf.TensorShape([None,])))
    iterator = dataset.make_one_shot_iterator()
    s, p = iterator.get_next()
    return {'s': s, 'p': p}


def evaluate_rank(model,
                  valid_triples,
                  test_triples,
                  all_triples,
                  e2idx,
                  p2idx):
    for eval_name, eval_triples in [('test', test_triples)]:
        _scores_o = list(model.predict(
            lambda: o_input_fn(eval_triples,
                               e2idx, 
                               p2idx)))
        ScoresO = np.reshape(_scores_o, [len(eval_triples), len(e2idx)])
        ranks_o, filtered_ranks_o = [], []
        for ((s, p, o), scores_o) in tqdm(zip(eval_triples, ScoresO),
                                          total=len(eval_triples),
                                          ncols=70):
            s_idx, p_idx, o_idx = e2idx[s], p2idx[p], e2idx[o]
            ranks_o += [1 + np.argsort(np.argsort(- scores_o))[o_idx]]
            filtered_scores_o = scores_o.copy()
            rm_idx_o = [e2idx[fo] for (fs, fp, fo) in all_triples if fs == s and fp == p and fo != o]
            filtered_scores_o[rm_idx_o] = - np.inf
            filtered_ranks_o += [1 + np.argsort(np.argsort(- filtered_scores_o))[o_idx]]
        for setting_name, setting_ranks in [('Raw', ranks_o), ('Filtered', filtered_ranks_o)]:
            mean_rank = np.mean(1 / np.asarray(setting_ranks))
            print('[{}] {} MRR: {}'.format(eval_name, setting_name, mean_rank))
            for k in [1, 3, 5, 10]:
                hits_at_k = np.mean(np.asarray(setting_ranks) <= k) * 100
                print('[{}] {} Hits@{}: {}'.format(eval_name, setting_name, k, hits_at_k))

In [5]:
def dam(x1, x2, is_training):
    with tf.variable_scope('transform'):
        ops = [tf.layers.BatchNormalization(),
               tf.layers.Dropout(Config.dropout_rate),
               tf.layers.Dense(Config.embed_dim),
               tf.layers.BatchNormalization(),]
        
        x1 = transform(x1, ops, is_training)
        x2 = transform(x2, ops, is_training)
    
    with tf.variable_scope('attend'):
        ops = [tf.layers.Dropout(Config.dropout_rate),
               tf.layers.Dense(Config.embed_dim, Config.activation),]
        
        x1_, x2_, align, align_t = attend(x1, x2, ops, is_training)
    
    with tf.variable_scope('compare'):
        ops = [tf.layers.BatchNormalization(),
               tf.layers.Dropout(Config.dropout_rate),
               tf.layers.Dense(Config.embed_dim, Config.activation),]
        
        x1 = compare(x1, x1_, ops, is_training)
        x2 = compare(x2, x2_, ops, is_training)
    
    with tf.variable_scope('aggregate'):
        ops = [tf.layers.BatchNormalization(),
               tf.layers.Dropout(Config.dropout_rate),
               tf.layers.Dense(Config.embed_dim, Config.activation),]
        
        x = aggregate(x1, x2, ops, is_training)
        x = tf.layers.batch_normalization(x, training=is_training)
        return x, align, align_t

    
def transform(x, ops, is_training):
    batch_norm_1, dropout, proj, batch_norm_2 = ops
    x = batch_norm_1(x, training=is_training)
    x = dropout(x, training=is_training)
    x = proj(x)
    x = batch_norm_2(x, training=is_training)
    return x


def attend(x1, x2, ops, is_training):
    def _transform(x):
        dropout, proj = ops
        x = dropout(x, training=is_training)
        x = proj(x)
        return x
    align = tf.matmul(_transform(x1), _transform(x2), transpose_b=True)
    align /= np.sqrt(Config.embed_dim)
    align_t = tf.transpose(align, [0, 2, 1])
    align = tf.nn.softmax(align)
    align_t = tf.nn.softmax(align_t)
    x1_ = tf.matmul(align, x2)
    x2_ = tf.matmul(align_t, x1)
    return x1_, x2_, align, align_t


def compare(x, x_, ops, is_training):
    batch_norm, dropout, proj = ops
    x = tf.concat([x, x_], axis=2)
    x = batch_norm(x, training=is_training)
    x = dropout(x, training=is_training)
    x = proj(x)
    return x


def aggregate(x1, x2, ops, is_training):
    batch_norm, dropout, proj = ops
    x = tf.concat([tf.reduce_sum(x1, axis=1),
                   tf.reduce_sum(x2, axis=1)], axis=1)
    x = batch_norm(x, training=is_training)
    x = dropout(x, training=is_training)
    x = proj(x)
    return x

In [6]:
def forward(features, mode, params):
    batch_sz = tf.shape(features['s'])[0]
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    
    e_embed = tf.get_variable('e_embed', [params['e_vocab_size'], Config.embed_dim])
    p_embed = tf.get_variable('p_embed', [params['p_vocab_size'], Config.embed_dim])
    
    s = tf.nn.embedding_lookup(e_embed, features['s'])
    p = tf.nn.embedding_lookup(p_embed, features['p'])
    
    query = context = tf.stack([s, p], axis=1)
    
    output, align, align_t = dam(query, context, is_training)
    
    logits = tf.matmul(output, e_embed, transpose_b=True)
    return logits, align, align_t
    
    
def model_fn(features, labels, mode, params):
    logits, align, align_t = forward(features, mode, params)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        log_params()
        
        loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                                         labels=labels))
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = tf.train.AdamOptimizer().minimize(loss_op,
                                                         global_step=tf.train.get_global_step())
        
        lth_1 = tf.train.LoggingTensorHook({'align': tf.reshape(align[0], [-1])},
                                           every_n_iter=100)
        lth_2 = tf.train.LoggingTensorHook({'align_t': tf.reshape(align_t[0], [-1])},
                                           every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(mode = mode,
                                          loss = loss_op,
                                          train_op = train_op,
                                          training_hooks = [lth_1, lth_2],)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions = tf.sigmoid(logits))


def log_params():
    tf.logging.info('\n'+pprint.pformat(tf.trainable_variables()))
    tf.logging.info('params: %d'%count_train_params())
    
    
def count_train_params():
    return np.sum([np.prod([d.value for d in v.get_shape()]) for v in tf.trainable_variables()])


def prt_epoch(n_epoch):
    print()
    print("EPOCH %d"%(n_epoch+1))
    print()

In [7]:
triples_all, triples_tr, triples_va, triples_te = load_triple()
e2idx, p2idx, params = build_vocab(triples_all)
sp2o = build_multi_label(triples_tr)

model = tf.estimator.Estimator(model_fn,
                               params = params)

for n_epoch in range(Config.n_epochs):
    prt_epoch(n_epoch)
    model.train(lambda: train_input_fn(triples_tr, e2idx, p2idx, sp2o))
    evaluate_rank(model,
                  triples_va,
                  triples_te,
                  triples_all,
                  e2idx,
                  p2idx,)

Files Already Downloaded
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpbsyqgy12', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1194c7e10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

EPOCH 1

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization/gamma:0' shape=(200,) dtype=float32_ref>,

100%|█████████████████████████████| 5000/5000 [01:50<00:00, 45.27it/s]


[test] Raw MRR: 0.017989010426362775
[test] Raw Hits@1: 0.8
[test] Raw Hits@3: 2.1
[test] Raw Hits@5: 2.44
[test] Raw Hits@10: 3.54
[test] Filtered MRR: 0.018362842204457847
[test] Filtered Hits@1: 0.8200000000000001
[test] Filtered Hits@3: 2.12
[test] Filtered Hits@5: 2.46
[test] Filtered Hits@10: 3.5999999999999996

EPOCH 2

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization/gamma:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization/beta:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/dense/kernel:0' shape=(200, 200) dtype=float32_ref>,
 <tf.Variable 'transform/dense/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization_1/gamma:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization_1/beta:0' shape=(200,) dtype=float32

100%|█████████████████████████████| 5000/5000 [01:50<00:00, 45.18it/s]


[test] Raw MRR: 0.023560353652747154
[test] Raw Hits@1: 1.58
[test] Raw Hits@3: 2.4
[test] Raw Hits@5: 2.68
[test] Raw Hits@10: 3.6999999999999997
[test] Filtered MRR: 0.025296947836951737
[test] Filtered Hits@1: 1.66
[test] Filtered Hits@3: 2.6
[test] Filtered Hits@5: 2.92
[test] Filtered Hits@10: 4.12

EPOCH 3

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization/gamma:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization/beta:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/dense/kernel:0' shape=(200, 200) dtype=float32_ref>,
 <tf.Variable 'transform/dense/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization_1/gamma:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization_1/beta:0' shape=(200,) dtype=float32_ref>,
 <tf.Va

...

[test] Raw MRR: 0.2723507764849665
[test] Raw Hits@1: 19.74
[test] Raw Hits@3: 30.28
[test] Raw Hits@5: 35.04
[test] Raw Hits@10: 41.120000000000005
[test] Filtered MRR: 0.3102600917113934
[test] Filtered Hits@1: 24.0
[test] Filtered Hits@3: 33.879999999999995
[test] Filtered Hits@5: 38.2
[test] Filtered Hits@10: 44.14

EPOCH 29

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization/gamma:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization/beta:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/dense/kernel:0' shape=(200, 200) dtype=float32_ref>,
 <tf.Variable 'transform/dense/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization_1/gamma:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization_1/beta:0' shape=(200,) dtype=floa

100%|█████████████████████████████| 5000/5000 [01:52<00:00, 44.55it/s]


[test] Raw MRR: 0.27557909089990523
[test] Raw Hits@1: 20.06
[test] Raw Hits@3: 30.48
[test] Raw Hits@5: 35.66
[test] Raw Hits@10: 41.24
[test] Filtered MRR: 0.3142250659016737
[test] Filtered Hits@1: 24.36
[test] Filtered Hits@3: 34.28
[test] Filtered Hits@5: 38.62
[test] Filtered Hits@10: 44.2

EPOCH 30

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization/gamma:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization/beta:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/dense/kernel:0' shape=(200, 200) dtype=float32_ref>,
 <tf.Variable 'transform/dense/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization_1/gamma:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'transform/batch_normalization_1/beta:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 

100%|█████████████████████████████| 5000/5000 [01:50<00:00, 45.05it/s]


[test] Raw MRR: 0.2761385819961074
[test] Raw Hits@1: 20.419999999999998
[test] Raw Hits@3: 30.099999999999998
[test] Raw Hits@5: 35.14
[test] Raw Hits@10: 41.88
[test] Filtered MRR: 0.31369441604540127
[test] Filtered Hits@1: 24.64
[test] Filtered Hits@3: 33.660000000000004
[test] Filtered Hits@5: 38.26
[test] Filtered Hits@10: 44.28
